In [3]:
#importing necessary libraries
import numpy as np 
import pandas as pd
from datetime import datetime
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt
import re

In [2]:
#importing data from Shopify Sales Report csv export
shopify_raw = pd.read_csv('sales_2016-01-01_2021-04-30.csv')

In [13]:
#shopify columns needed in Sales Report
#note api_client_title is in fact sales channel within shopify system
'''
['order_id', 'order_name',
'api_client_title', 'hour', 'customer_id', 'customer_email',
'customer_type', 'orders', 'gross_sales', 'discounts', 'returns',
'net_sales', 'shipping', 'taxes', 'total_sales',
'ordered_item_quantity', 'returned_item_quantity', 'net_quantity',
'''

In [74]:
#converting date to datetime & adding second date column
shopify_raw['hour'] = pd.to_datetime(shopify_raw.hour)
shopify_raw['hour_2']= shopify_raw['hour']

In [75]:
#forming orders table
orders = shopify_raw.groupby(['order_id','order_name','customer_id']).agg({'hour':'min','gross_sales':'sum','discounts':'sum','returns':'sum','net_sales':'sum','total_sales':'sum','customer_type':'min','api_client_title':'first'}).reset_index()
orders.set_index('order_id',inplace=True)

In [76]:
#forming customers table
customers = shopify_raw.groupby(['customer_id','customer_email']).agg({'hour':'min','hour_2':'max','order_id':'count','total_sales':'sum','ordered_item_quantity':'sum','returned_item_quantity':'sum','api_client_title':'first'})
customers = customers.reset_index()

In [77]:
#AOV for each customer
customers['AOV'] = customers['total_sales']/customers['order_id']

In [78]:
#orders in the last year, update with desired date
desired_date = '2020-04-30 00:00:00'
orders_last_year = orders[orders['hour'] > desired_date]
orders_last_year = orders_last_year.reset_index()

In [79]:
#AOV & Freq for all orders in the last year
aov = orders_last_year['total_sales'].sum()/orders_last_year['order_id'].nunique()
freq = orders_last_year['order_id'].nunique()/orders_last_year['customer_id'].nunique()

In [80]:
#Bringing it together to get customer value
customer_value_last_year = freq * aov

In [81]:
#survival analysis to get retention rate, let's prep the data
#we need 3 things: 
#1) occurence of an event (churn)
#2) field that represents passage of time (tenure) 
#3) a field to group customers by (here we are going to do for store as a whole
#but we could compare different marketing or sales channels for example)
#note hour_2 denotes date of last order

customers=customers.reset_index()
customers['tenure'] = customers['hour_2']-customers['hour']
customers['tenure'] = customers['tenure'].dt.days
customers['today'] = datetime.now()
customers['days_since_last_order'] = (customers['today'] - customers['hour_2']).dt.days
customers['group'] = 'company_customer'
#note period defining churn will depend on business in question
customers['churn?'] = customers['days_since_last_order'] >= 260


In [82]:
#defining survival function
def survival(data,group_field, time_field, event_field):
    model = KaplanMeierFitter()
    results = []
    
    for i in data[group_field].unique():
        group = data[data[group_field]==i]
        T = group[time_field]
        E = group[event_field]
        model.fit(T, E, label=str(i))
        results.append(model.survival_function_)
    
    survival = pd.concat(results, axis=1)
    return survival

In [84]:
#applying it to different sales 
rates = survival(customers[customers['hour'] > desired_date], 'group', 'tenure', 'churn?')

In [ ]:
#graphing it
plt.plot(rates)
plt.legend(labels = customers['group'].value_counts().index,loc="upper right")
plt.ylabel('Retention Rate')
plt.xlabel('Tenure')
plt.show()

In [103]:
#store 1 year retention rate as variable
retention_rate = rates.loc[365][0]

In [108]:
#churn is 1-retention rate
churn_rate = 1-retention_rate

In [109]:
#lifespan is 1/churn rate in the period it is measured
customer_lifespan = 1/churn_rate

In [110]:
#CLTV is customer value * lifespan
cltv = customer_value_last_year*customer_lifespan

In [112]:
#you can repeat survival analysis to compare retention across customers with different characteristics
